In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dotenv import load_dotenv
load_dotenv('../.env')

In [ ]:
from autodm.llm import get_llm
from autodm.roll import Dice, DiceType
from autodm.character import Character
from autodm.adventurelog import AdventureLog

from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.agent import ReActAgent
from llama_index.core.llms import ChatMessage
import typing as T
import gradio as gr

from typing import List, Union
from rich import print

In [ ]:
character = Character.generate(name='Aldrin', level=1)

In [ ]:
character.spells = ['magic missile', 'light']
character.equipment = ['simple wand', 'simple robe']

In [ ]:
print(character)

In [ ]:
log = AdventureLog()
log.clear()
log = AdventureLog()
opening = f"""\
You are a {character.chr_race} {character.chr_class} named {character.name}.\
"""
print(opening)

log.add_entry(opening)

adventure_log_tool = QueryEngineTool.from_defaults(
    query_engine=log.index.as_query_engine(llm=get_llm()), 
    description="Use this tool when you want to look up anything that has happened in the game.",
    name='adventure_log_tool'
)

In [ ]:
def roll(dice_type: Union[int, DiceType] = DiceType.D20, dice_count: int = 1) -> str:
    "Rolls 1 or more die. Dice can be of type D4, D6, D8, D10, D12, or D20. \
You can also roll multiple dice at once, and the result is the sum of the rolls."
    dice = Dice(type=DiceType(dice_type), count=dice_count)
    return f"Rolling {dice_count}d{dice_type}. Result: {dice.roll()}"

roll_tool = FunctionTool.from_defaults(roll, description="Use this tool when you want to roll dice to determine the outcome of an action.")

In [ ]:
agent = ReActAgent.from_tools(tools=[adventure_log_tool, roll_tool], llm=get_llm(), verbose=1)
pt = agent.get_prompts()['agent_worker:system_prompt']
character_str = f"The player's character is a {character.chr_race} {character.chr_class} of level {character.level} named {character.name}. \
The character has {character.hp} out of {character.max_hp} HP, and the following attributes: {character.attributes}. "

new_template = """\
You are a talented D&D dungeon master whose goal is to walk the player through a D&D campaign. \
When the player asks you a question, you should use the tools at your disposal to answer the question. \
If the player takes an action, make sure the result is in line with D&D rules and preveious events in the adventure log. \
If there is no tool available to answer the question, you create exciting and funny experiences for the player. \
Try to match the player's tone and style to keep them engaged. \
Also make sure to regularly progress the story by providing the player with new information. \
You may also roll dice for the player when appropriate, including attack rolls, saving throws, and skill checks. \
%s 

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand. \
This may require breaking the task into subtasks and using different tools to complete each subtask. \
Once you complete the task, you will provide the answer to the player. \
Do not take additional actions on behalf of the player unless instructed to do so - for example, if they say "look around," describe what they see according to the detail of a perception roll. \
Do not take any additional steps. \

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in English using the following format:

```
Thought: I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do this {{'input': 'hello world', 'num_beams': 5}}.

If this format is used, the user will respond in the following format:

```
Observation: tool response
```

You should keep repeating the above format until you have enough information to answer the question without using 
any more tools. At that point, you MUST respond in the one of the following two formats:

```
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 
```

```
Thought: I cannot answer the question with the provided tools. I will create an engaging experience for the player.
Answer: 
```

## Current Conversation

Below is the current conversation consisting of interleaving human and assistant messages.\
""" % character_str

pt.template = new_template
agent.update_prompts({'agent_worker:system_prompt': pt})

In [ ]:
def msg(message: str, user='player') -> str:
    log.add_entry(message, user=user)
    response = agent.chat(message)
    log.add_entry(response.response, user='dm')
    return response.response

In [ ]:
resp = msg("Create an engaging opening for a D&D game by describing how the player enters a new and uncharted territory.", 'dm')

In [ ]:
import gradio as gr

In [ ]:
def respond(message, chat_history):
    bot_message = msg(message)
    chat_history.append((message, bot_message))
    return '', chat_history

In [ ]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(value=[("", resp)])
    message = gr.Textbox()
    clear = gr.ClearButton([message, chatbot])

    message.submit(respond, [message, chatbot], [message, chatbot])

demo.launch(inline=False)